In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import os

from obspy import read

In [2]:
time_arr = np.linspace(0.0, 30.0, 3001)
time_arr.shape

(3001,)

In [3]:
# plot wave normal
def plot_wave_processed(data):
    fig = plt.figure(figsize=(15, 4))
    ax = fig.add_subplot(1, 1, 1)
    cmap = plt.get_cmap('gnuplot')
    colors = [cmap(i) for i in np.linspace(0, 1, len(data))]
    i = 0
    for s in data:
        ax.plot(time_arr, s, color=colors[i])
        i+=1
    ax.tick_params(axis='both', labelsize=15)
    ax.tick_params(axis='both', labelsize=15)
    plt.ylabel('HHZ Velocity', fontsize=20)
    plt.xlabel('Timestep', fontsize=20);
    plt.grid()
    plt.show()
    del fig
    del ax

In [10]:
os.getcwd()

'C:\\Programming\\Earthquake\\datasets'

In [11]:
os.chdir('../')
os.getcwd()

'C:\\Programming\\Earthquake'

In [12]:
os.chdir('datasets/')

In [7]:
# READING EARTHQUAKE BEHAVIOUR
pkl_file = open('../datasets/100hz/earthquakes_seismic_100hz.pkl', 'rb')
earthquakes_data = pickle.load(pkl_file)
pkl_file.close()

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/100hz/earthquakes_seismic_100hz.pkl'

In [13]:
# READING NORMAL BEHAVIOUR
pkl_file = open('../datasets/100hz/normal_seismic_100hz.pkl', 'rb')
normal_data = pickle.load(pkl_file)
pkl_file.close()

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/100hz/normal_seismic_100hz.pkl'

In [7]:
(len(earthquakes_data), len(earthquakes_data[0]), len(earthquakes_data[0][0]))

(1394, 58, 3001)

In [8]:
(len(normal_data), len(normal_data[0]), len(normal_data[0][0]))

(1074, 58, 3001)

In [9]:
def check_nans_incomplete_stations(data):
    
    # check for possible nans and incomplete station data
    nan_arr = []
    not_58_stations = []
    not_61_samples = []

    for i in range(0, len(data)):
        
        try:
            # stations != 58 per event
            if len(data[i]) != 58:
                not_58_stations.append(i)
                
            for j in range(0, 58):
                
                try:
                    if len(data[i][j]) != 3001:
                        not_61_samples.append(i)
                # a station has nan value(s)
                    
                    is_nan_event = np.isnan(np.sum(data[i][j]))
                    if is_nan_event:
                        nan_arr.append(i)
                    
                except:
                    not_61_samples.append(i)
                
        except:
            not_58_stations.append(i)
            
        
                
    return nan_arr, not_58_stations, not_61_samples

# verify that we only have one shape for stations and samples
def verify_unique_shapes(data):
    stations_shape = [58]
    samples_shape = [3001]
    
    for i in data:
        station_shape = len(i)
    
        if station_shape not in stations_shape:
            stations_shape.append(station_shape)
            
        for j in i:
            sample_shape = len(j)
            if sample_shape not in samples_shape:
                samples_shape.append(sample_shape)
    
    return stations_shape, samples_shape


def check_nans_incomplete_stations_preprocessed(data):
    # check for possible nans and incomplete station data for lstm processed input
    nan_arr = []
    not_58_stations = []
    not_61_samples = []

    for i in range(0, len(data)):
        
        try:
            # samples != 61 per station
            if len(data[i]) != 151:
                not_61_samples.append(i)
                
            for j in range(0, 3001):
                
                try:
                    if len(data[i][j]) != 58:
                        not_58_stations.append(i)
                # a station has nan value(s)
                    
                    is_nan_event = np.isnan(np.sum(data[i][j]))
                    if is_nan_event:
                        nan_arr.append(i)
                    
                except:
                    not_58_stations.append(i)
                
        except:
            not_61_samples.append(i)
            
        
                
    return nan_arr, not_58_stations, not_61_samples

# verify that we only have one shape for stations and samples for lstm processed input
def verify_unique_shapes_preprocessed(data):
    stations_shape = [58]
    samples_shape = [3001]
    
    for i in data:
     
        sample_shape = len(i)
        if sample_shape not in samples_shape:
            samples_shape.append(sample_shape)
        
        for j in i:
            station_shape = len(j)
            if station_shape not in stations_shape:
                stations_shape.append(station_shape)


    
    return stations_shape, samples_shape

In [10]:
nan_arr_e, not_58_stations_e, not_61_samples_e = check_nans_incomplete_stations(earthquakes_data)

In [11]:
len(nan_arr_e), len(not_58_stations_e), len(not_61_samples_e)

(0, 0, 16)

In [12]:
remove_indices = [y for x in [nan_arr_e, not_58_stations_e] for y in x]
remove_indices = [y for x in [remove_indices, not_61_samples_e] for y in x]
remove_indices = set(remove_indices)
remove_indices = list(remove_indices)
remove_indices

[481,
 965,
 1197,
 110,
 463,
 878,
 1233,
 18,
 914,
 54,
 1174,
 56,
 505,
 186,
 1339,
 414]

In [13]:
earthquakes_data_processed = earthquakes_data
earthquakes_data_processed = [i for j, i in enumerate(earthquakes_data_processed) if j not in remove_indices]

In [14]:
len(earthquakes_data_processed)

1378

In [15]:
nan_arr_e, not_58_stations_e, not_61_samples_e = check_nans_incomplete_stations(earthquakes_data_processed)

In [16]:
len(nan_arr_e), len(not_58_stations_e), len(not_61_samples_e)

(0, 0, 0)

In [17]:
verify_unique_shapes(earthquakes_data_processed)

([58], [3001])

In [18]:
(len(earthquakes_data_processed), len(earthquakes_data_processed[0]), len(earthquakes_data_processed[0][0]))

(1378, 58, 3001)

In [19]:
nan_arr_n, not_58_stations_n, not_61_samples_n = check_nans_incomplete_stations(normal_data)

In [20]:
len(nan_arr_n), len(not_58_stations_n), len(not_61_samples_n)

(0, 0, 11)

In [21]:
remove_indices = [y for x in [nan_arr_n, not_58_stations_n] for y in x]
remove_indices = [y for x in [remove_indices, not_61_samples_n] for y in x]
remove_indices = set(remove_indices)
remove_indices = list(remove_indices)
remove_indices

[451, 967, 107, 363, 75, 238, 684, 304, 924, 510, 671]

In [22]:
normal_data_processed = normal_data
normal_data_processed = [i for j, i in enumerate(normal_data_processed) if j not in remove_indices]

In [23]:
nan_arr_n, not_58_stations_n, not_61_samples_n = check_nans_incomplete_stations(normal_data_processed)

In [24]:
len(nan_arr_n), len(not_58_stations_n), len(not_61_samples_n)

(0, 0, 0)

In [25]:
verify_unique_shapes(normal_data_processed)

([58], [3001])

In [26]:
len(normal_data_processed)

1063

In [27]:
# same length as normal
earthquakes_data_processed = earthquakes_data_processed[0:500]
normal_data_processed = normal_data_processed[0:500]

In [28]:
len(earthquakes_data_processed), len(normal_data_processed)

(500, 500)

In [29]:
# turns data to lstm input for training
def to_lstm_input(read_file, label):
    lstm_event_arr = []
    for i, r in enumerate(read_file):
        arr = read_file[i]
        arr_t = np.transpose(arr)
        try:
            arr_t = arr_t.reshape(3001, 58)
        except:
            continue
        lstm_event_arr.append({label: arr_t})
    lstm_event_arr = np.array(lstm_event_arr)
    return lstm_event_arr

In [30]:
earthquakes_data_processed = np.array(earthquakes_data_processed)

In [31]:
normal_data_processed = np.array(normal_data_processed)

In [32]:
earthquakes_data_processed.shape, normal_data_processed.shape

((500, 58, 3001), (500, 58, 3001))

In [33]:
# earthquakes == 1
lstm_input_earthquakes = to_lstm_input(earthquakes_data_processed, '1')

In [34]:
# normal == 0
lstm_input_normal = to_lstm_input(normal_data_processed,'0')

In [35]:
lstm_input_earthquakes.shape, lstm_input_normal.shape

((500,), (500,))

In [36]:
lstm_input_earthquakes[0]

{'1': array([[-0.03062985, -0.0405771 ,  0.5747331 , ...,  0.10832102,
         -0.01657085, -0.06644579],
        [-0.03537532, -0.03877367,  0.57058126, ...,  0.12161497,
         -0.02874535, -0.12019142],
        [-0.04314064, -0.06221821,  0.56109134, ...,  0.13392418,
         -0.03922895, -0.17909074],
        ..., 
        [ 0.58196721,  0.00811542, -0.39857651, ...,  0.06597735,
         -0.31078796,  0.16841524],
        [ 0.57377049,  0.04147881, -0.39976275, ...,  0.06105367,
         -0.31518431,  0.10583471],
        [ 0.56988783,  0.0748422 , -0.39679715, ...,  0.05563762,
         -0.32600609,  0.19344745]])}

In [37]:
lstm_input = np.concatenate((lstm_input_earthquakes, lstm_input_normal))
lstm_input.shape

(1000,)

In [38]:
# shuffle the data set
np.random.shuffle(lstm_input)

In [39]:
#(70 training - 20 validation - 10 training split)
training_set = lstm_input[0:700]
validation_set = lstm_input[700:900]
test_set = lstm_input[900:1000]

In [40]:
x_train = []
y_train = []
for i, r in enumerate(training_set):
    arr = training_set[i]
    key = list(training_set[i].keys())[0]
    if key == '1':
        y_train.append(1)
    else:
        y_train.append(0)
    
    x_train.append(arr[key])
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [41]:
x_train.shape, y_train.shape

((700, 3001, 58), (700,))

In [42]:
os.chdir('../100HZ')
os.getcwd()

FileNotFoundError: [Errno 2] No such file or directory: '../100HZ'

In [119]:
pickle.dump(x_train, open("x_train.pkl", "wb"))
pickle.dump(y_train, open("y_train.pkl", "wb"))

In [120]:
len(np.where(y_train == 0)[0]), len(np.where(y_train == 1)[0]), len(np.where(y_train == 0)[0]) / len(y_train), len(np.where(y_train == 1)[0]) / len(y_train)

(725, 763, 0.48723118279569894, 0.5127688172043011)

In [121]:
x_validation = []
y_validation = []
for i, r in enumerate(validation_set):
    arr = validation_set[i]
    key = list(validation_set[i].keys())[0]
    if key == '1':
        y_validation.append(1)
    else:
        y_validation.append(0)
    
    x_validation.append(arr[key])
    
x_validation = np.array(x_validation)
y_validation = np.array(y_validation)

In [122]:
x_validation.shape, y_validation.shape

((425, 3001, 58), (425,))

In [123]:
len(np.where(y_validation == 0)[0]), len(np.where(y_validation == 1)[0]), len(np.where(y_validation == 0)[0]) / len(y_validation), len(np.where(y_validation == 1)[0]) / len(y_validation)


(239, 186, 0.5623529411764706, 0.4376470588235294)

In [124]:
pickle.dump(x_validation, open("x_validation.pkl", "wb"))
pickle.dump(y_validation, open("y_validation.pkl", "wb"))

In [125]:
x_test = []
y_test = []
for i, r in enumerate(test_set):
    arr = test_set[i]
    key = list(test_set[i].keys())[0]
    if key == '1':
        y_test.append(1)
    else:
        y_test.append(0)
    
    x_test.append(arr[key])
    
x_test = np.array(x_test)
y_test = np.array(y_test)

In [126]:
x_test.shape, y_test.shape

((213, 3001, 58), (213,))

In [127]:
len(np.where(y_test == 0)[0]), len(np.where(y_test == 1)[0]), len(np.where(y_test == 0)[0]) / len(y_test) , len(np.where(y_test == 1)[0]) / len(y_test)

(99, 114, 0.4647887323943662, 0.5352112676056338)

In [128]:
pickle.dump(x_test, open("x_test.pkl", "wb"))
pickle.dump(y_test, open("y_test.pkl", "wb"))

In [ ]:
data = [[len(np.where(y_train == 0)[0]) / len(y_train),
        len(np.where(y_validation == 0)[0]) / len(y_validation),
        len(np.where(y_test == 0)[0]) / len(y_test)],
        
       [len(np.where(y_train == 1)[0]) / len(y_train),
        len(np.where(y_validation == 1)[0]) / len(y_validation),
        len(np.where(y_test == 1)[0]) / len(y_test)]]

X = np.arange(3)
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0,0,1,1])
ax.bar(X + 0.00, data[0], color = 'g', width = 0.25)
ax.bar(X + 0.25, data[1], color = 'r', width = 0.25)
colors = {'normal behaviour':'green', 'earthquake':'red'}  
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels)
plt.title("The Class Ratio Between Different Datasets", fontsize=16)
plt.ylabel('Percentage (%)', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)
plt.xticks([])

In [ ]:
x_train = pickle.load(open("../datasets/5hz/x_train.pkl", "rb"))
y_train = pickle.load(open("../datasets/5hz/y_train.pkl", "rb"))

x_validation = pickle.load(open("../datasets/5hz/x_validation.pkl", "rb"))
y_validation = pickle.load(open("../datasets/5hz/y_validation.pkl", "rb"))

x_test = pickle.load(open("../datasets/5hz/x_test.pkl", "rb"))
y_test = pickle.load(open("../datasets/5hz/y_test.pkl", "rb"))

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_validation.shape, y_validation.shape

In [ ]:
x_test.shape, y_test.shape

In [ ]:
nan_arr_n, not_58_stations_n, not_61_samples_n = check_nans_incomplete_stations_preprocessed(x_train)
len(nan_arr_n), len(not_58_stations_n), len(not_61_samples_n)

In [ ]:
verify_unique_shapes_preprocessed(x_train)

In [ ]:
nan_arr_n, not_58_stations_n, not_61_samples_n = check_nans_incomplete_stations_preprocessed(x_validation)
len(nan_arr_n), len(not_58_stations_n), len(not_61_samples_n)

In [ ]:
verify_unique_shapes_preprocessed(x_validation)

In [ ]:
nan_arr_n, not_58_stations_n, not_61_samples_n = check_nans_incomplete_stations_preprocessed(x_test)
len(nan_arr_n), len(not_58_stations_n), len(not_61_samples_n)

In [ ]:
verify_unique_shapes_preprocessed(x_test)

In [7]:
os.getcwd()

'/Volumes/My Passport'

In [4]:
os.chdir('../../../../../')

In [6]:
os.chdir('Volumes/My Passport')

In [17]:
e = pickle.load(open("normal_seismic_50hz.pkl", "rb"))

In [18]:
e.shape

(20756, 58, 1501)

In [19]:
en = e[:1000]

In [20]:
en.shape

(1000, 58, 1501)

In [21]:
pickle.dump(en, open("n_normal_sesimic_50hz.pkl", "wb"))